# Introdução

Neste documento realizamos os embeddings de cada trecho formatado anteriormente, incluindo o titulo e o resumo.

Esses embeddings serão salvos em uma vectorstore.

In [1]:
# !pip install langchain==0.0.340 langchain --force-reinstall

# Obtenção dos Metadados do Minerva

In [2]:
import os
import json

In [3]:
pdf_folder_path = "./metadata_documents"

## Arquivos de Metadados

In [4]:
filename_list = []
for filename in os.listdir(pdf_folder_path):
    if filename.endswith(".json") and not filename.endswith("_x.json"):
        filename_list.append(os.path.join(pdf_folder_path, filename))

filename_list = sorted(filename_list)

In [5]:
filename_list [:5]

['./metadata_documents/documento_.json',
 './metadata_documents/documento_276189.json',
 './metadata_documents/documento_494080.json',
 './metadata_documents/documento_547235.json',
 './metadata_documents/documento_614469.json']

## Obtenção dos Metadados

In [6]:
def get_metadata(filename_list):
    metadata_dict = {}
    
    for filename in filename_list:
        with open(filename, "r") as file: 
            metadata = json.load(file)
            _id = metadata["id"]
            metadata_dict[_id] = metadata
    
    return metadata_dict

In [7]:
metadata_dict = get_metadata(filename_list)

In [8]:
len(metadata_dict)

2042

In [9]:
metadata_dict["614469"].keys()

dict_keys(['title', 'authors', 'abstract', 'bibliography_pages', 'keywords', 'urls', 'pdf_url', 'id'])

In [10]:
metadata_dict["614469"]["title"]

'Evolução do relevo, coberturas sedimentares e formação de solos em superfícies geomorfológicas : Medio Vale do Rio Paraíba do Sul (SP/RJ)'

In [11]:
metadata_dict["614469"]["abstract"][:100]

'Apesar de o radiocarbono ser utilizado na cronologia dos depósitos holocênicos e pleistocênicos na r'

# Parte 0: Instalações

In [12]:
# %pip install faiss-gpu

# Parte 1: Vector Store

In [13]:
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.vectorstores import FAISS

In [14]:
from langchain.embeddings import SentenceTransformerEmbeddings

In [15]:
embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

/home/vinic/miniconda3/envs/llama/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
vectorstore = FAISS.load_local("./new_faiss_vector_store/", embeddings)

# Parte 2: Vector Search

In [17]:
buscas = [
    "modelos de otimização focados em parques de energia eólica",
    "algoritmos de aprendizado de maquina para resolver problemas NP-Completos",
    "redes neurais complexas no contexto de biologia",
    "pesquisa a respeito dos efeitos da covid na memoria de longo prazo",
    "sistemas de large language models",
    "novas espécies de dinossauros",
    "os efeitos de substancias alucinogenas no cerebro de jovens",
    "Evolução do relevo, coberturas sedimentares e formação de solos em superfícies geomorfológicas",
    "pesquisa sobre a comunicação persuasiva através da tecnologia, utilizando engenharia social",
    "estudo sobre o desenvolvimento de transtornos mentais em profissionais da medicina",
    "uso de algoritmos de aprendizado de máquina para identificação e prevenção de ataques em redes de computadores",
    "produção de epicloridrina em larga escala a partir do glicerol como substância base",
    "estudos sobre saúde mental em alunos de universidades públicas no campo da psicologia social"
]

In [18]:
query = buscas[9]
query

'estudo sobre o desenvolvimento de transtornos mentais em profissionais da medicina'

## Proximidade de Titulos

In [19]:
# !pip install langchain==0.0.340

In [20]:
print("Query:", query, "\n")

res = vectorstore.similarity_search_with_score(
    query,
    k=10,
    fetch_k=100000,
    filter=dict(type="title")
)
len(res)
[print(r[1], ":", r[0].metadata["id"], ":", r[0].page_content) for r in res]

Query: estudo sobre o desenvolvimento de transtornos mentais em profissionais da medicina 

2.960468 : 920863 : Inovações práticas e pistas para formação médica no campo da saúde mental : a passagem pelo internato em Carmo - RJ
3.4189978 : 916013 : Prevalência de transtornos mentais no estudantes de medicina da Universidade Federal do Rio de Janeiro
3.7175002 : 889637 : Estudo do estigma como barreira ao tratamento dos transtornos mentais no contexto de uma intervenção psicossocial
4.102117 : 894489 : Muros e passagens : práticas assistenicais na internação psiquiátrica em hospital especializado
4.456772 : 921772 : Fisioterapia e inovação : Atenção psicossocial, motivação e o cuidado neurocognitivo funcional de idosos com transtornos mentais
4.9555397 : 916084 : Religiosidade e saúde mental : influências na experiência de tratamento de portadores de transtorno psíquico
4.9571075 : 917937 : Aplicação e validação de instrumentos psicométricos na psiquiatria forense : importância clínica 

[None, None, None, None, None, None, None, None, None, None]

## Proximidade de Resumos

In [21]:
print("Query:", query, "\n")

res = vectorstore.similarity_search_with_score(
    query,
    k=5,
    fetch_k=100000,
    filter=dict(type="abstract")
)
len(res)

for r in res:
    print(r[1], ":", r[0].metadata["id"], ":", r[0].page_content, "\n")

Query: estudo sobre o desenvolvimento de transtornos mentais em profissionais da medicina 

4.0081997 : 931498 : A Reforma Psiquiátrica Brasileira permitiu problematizar o lugar que o indivíduo diagnosticado com algum transtorno psiquiátrico ocupa na sociedade contemporânea, repensar saberes e práticas na assistência à saúde mental, e institucionalizar no contexto brasileiro um modelo de assistência que se contrapõe ao à internação psiquiátrica, admitida como medida excepcional, e uma intervenção terapêutica de base comunitária visando o restabelecimento deste indivíduo à vida social. A desinstitucionalização inicia-se como um aspecto central com o objetivo de superar o modelo asilar e permitir às pessoas advindas de longas internações a vida fora das instituições sua inclusão social. Concomitantemente é criado um sistema de assistência social, com programas de transferência de renda e outros benefícios sociais, garantindo apoio social como um direito de cidadania a população economica

## Proximidade de Frases

In [22]:
print("Query:", query, "\n")

res = vectorstore.similarity_search_with_score(
    query,
    k=100,
    fetch_k=1000000,
    filter=dict(type="sentences")
)

Query: estudo sobre o desenvolvimento de transtornos mentais em profissionais da medicina 



In [23]:
len(res)

for r in res[:10]:
    print(r[1], ":", r[0].metadata["id"], ":", r[0].page_content)

2.8952703 : 916084 :  LIMA, M
 F
, FERREIRA, C
 B
 Estratégias de enfrentamento de pacientes com transtornos mentais
2.9416125 : 916084 :  M
, FUREGATO, A
 R
 F
 Relação de ajuda com paciente psiquiátrico: além do paradigma médico
2.9912298 : 916084 :  Estratégias de enfrentamento de pacientes com transtornos mentais
 Pesquisas e Práticas Psicossociais, São João Del Rei, v
 13, n
 2, p
 1-15, 2018
3.0091233 : 889637 : Um número crescente de estudos avaliou o estigma relacionado aos transtornos mentais e suas consequências. Alguns deles se concentraram nas atitudes do público e m relação às pessoas com transtornos mentais (KNIFTON et al, 2010; SCHOMERUS et al, 2012) , outros consideraram a percepção das famílias, dos cuidadores e dos profissionais, e outros avaliaram a experiência pessoal daqueles que sofrem estigma (WAHL, 1999; STUART, 2005; King et al, 2007) . Há, portanto, polissemia nos proces sos de estigmatização sofridos por portadores de transtornos mentais na percepção e na viv

In [24]:
results = {}

# geramos um mapa com id.start_index.[lista dos documents que iniciam naquele index]
for doc in res:
    _id = doc[0].metadata["id"]
    _start_index = doc[0].metadata["start_index"]
    
    if _id not in results.keys():
        results[_id] = {}

    if _start_index not in results[_id].keys():
        results[_id][_start_index] = []
    
    results[_id][_start_index].append(doc)

In [25]:
results.keys()

dict_keys(['916084', '889637', '916013', '920863', '917904', '921772', '889610', '918182', '924713', '924999', '895776', '924254', '894489', '921103', '916920', '814014', '922622', '494080', '887841', '921122', '925329', '940138', '925204', '895957', '917938', '920719'])

In [26]:
# results

In [27]:
# a partir disso podemos selecionar apenas para cada start_index o de maior janela

results_by_document = {}
for _id, docs in results.items():
    
    if len(docs) <= 2:
        print("Documento", _id, "ignorado por pouco conteúdo similar.")
        continue
    
    results_by_document[_id] = []
    
    for start_index, doc_tuples in docs.items():
        elements = sorted(doc_tuples, key=lambda x: (x[1], -x[0].metadata["window_size"]), reverse=False)
        results_by_document[_id].append((elements[0][1], elements[0][0]))

Documento 921772 ignorado por pouco conteúdo similar.
Documento 889610 ignorado por pouco conteúdo similar.
Documento 918182 ignorado por pouco conteúdo similar.
Documento 924713 ignorado por pouco conteúdo similar.
Documento 895776 ignorado por pouco conteúdo similar.
Documento 916920 ignorado por pouco conteúdo similar.
Documento 814014 ignorado por pouco conteúdo similar.
Documento 922622 ignorado por pouco conteúdo similar.
Documento 494080 ignorado por pouco conteúdo similar.
Documento 887841 ignorado por pouco conteúdo similar.
Documento 921122 ignorado por pouco conteúdo similar.
Documento 925329 ignorado por pouco conteúdo similar.
Documento 940138 ignorado por pouco conteúdo similar.
Documento 925204 ignorado por pouco conteúdo similar.
Documento 895957 ignorado por pouco conteúdo similar.
Documento 917938 ignorado por pouco conteúdo similar.
Documento 920719 ignorado por pouco conteúdo similar.


In [28]:
for _id in results_by_document.keys():
    print(">> ", _id, ":", metadata_dict[_id]["title"], ":", len(results_by_document[_id]), "\n")
    
    # for doc in results_by_document[_id]:
    #     print(f"{doc.metadata['start_index']}\t{doc.metadata['window_size']}\t", doc.page_content, "\n")

>>  916084 : Religiosidade e saúde mental : influências na experiência de tratamento de portadores de transtorno psíquico : 9 

>>  889637 : Estudo do estigma como barreira ao tratamento dos transtornos mentais no contexto de uma intervenção psicossocial : 13 

>>  916013 : Prevalência de transtornos mentais no estudantes de medicina da Universidade Federal do Rio de Janeiro : 11 

>>  920863 : Inovações práticas e pistas para formação médica no campo da saúde mental : a passagem pelo internato em Carmo - RJ : 12 

>>  917904 : Percepção da doença no transtorno obsessivo-compulsivo, transtorno da ansiedade social e trnastorno de pânico : 6 

>>  924999 : Avaliação do comportamento suicida em estudantes de medicina da Universidade Federal do Rio de Janeiro : 4 

>>  924254 : O processo de constituição de redes comunitárias de apoio e inclusão para pessoas com transtorno mental na cidade do Rio de Janeiro : a experiência do programa Entrelaços : 4 

>>  894489 : Muros e passagens : práti

In [29]:
# results_by_document

## Retrieval Class

In [30]:
class Retriever:
    
    def __init__(self, vectorstore):
        self.vectorstore = vectorstore
    
    def _group_documents(self, retrieved_documents: list):
        """Groups documents by id"""
        
        results = {}

        # geramos um mapa com id.start_index.[lista dos documents que iniciam naquele index]
        for doc in retrieved_documents:
            _id = doc[0].metadata["id"]
            _start_index = doc[0].metadata["start_index"]

            if _id not in results.keys():
                results[_id] = {}

            if _start_index not in results[_id].keys():
                results[_id][_start_index] = []

            results[_id][_start_index].append(doc)
        
        return results
    
    def _organize_documents(self, grouped_documents: dict):
        """
        Organizes documents by better similarity results.
        Chooses longest document when multiple start on the same paragraph.
        Ignores documents with less than 2 results.
        """

        results_by_document = {}
        for _id, docs in grouped_documents.items():

            if len(docs) <= 2:
                # print("Documento", _id, "ignorado por pouco conteúdo similar.")
                continue

            results_by_document[_id] = []

            for start_index, doc_tuples in docs.items():
                # a partir disso podemos selecionar apenas para cada start_index o de maior janela
                elements = sorted(doc_tuples, key=lambda x: (x[1], -x[0].metadata["window_size"]), reverse=False)
                results_by_document[_id].append((elements[0][1], elements[0][0]))
        
        return results_by_document

    def _limit_k_by_document(self, organized_documents, max_k_by_document):
        organized_documents = organized_documents.copy()
        
        for _id in organized_documents.keys():
            doc_tuples = organized_documents[_id]
            sorted_by_score = sorted(doc_tuples, key=lambda x: x[0], reverse=False)
            top_k_sorted_by_score = sorted_by_score[:max_k_by_document]
            
            organized_documents[_id] = top_k_sorted_by_score
            
        return organized_documents

    async def search(self, query, k, fetch_k, max_k_by_document=100, filter=None):
        res = vectorstore.similarity_search_with_score(
            query,
            k=k,
            fetch_k=fetch_k,
            filter=filter
        )
        
        grouped_documents = self._group_documents(res)
        organized_documents = self._organize_documents(grouped_documents)
        top_k_documents = self._limit_k_by_document(organized_documents, max_k_by_document)
        
        return top_k_documents

In [31]:
retriever = Retriever(vectorstore)

In [32]:
query = buscas[9]
query

'estudo sobre o desenvolvimento de transtornos mentais em profissionais da medicina'

In [33]:
retrieved_documents = await retriever.search(query, k=100, fetch_k=10000, max_k_by_document=10, filter={"type":"sentences"})

In [34]:
for _id in retrieved_documents.keys():
    print(">> ", _id, ":", metadata_dict[_id]["title"], ": Total", len(retrieved_documents[_id]), "\n")
    
    for doc in retrieved_documents[_id]:
        print(f"\tScore {str(round(doc[0], 3))}: Start Index [{doc[1].metadata['start_index']}] with window size [{doc[1].metadata['window_size']}]\n")

>>  916084 : Religiosidade e saúde mental : influências na experiência de tratamento de portadores de transtorno psíquico : Total 9 

	Score 2.895: Start Index [1265] with window size [5]

	Score 2.942: Start Index [1521] with window size [5]

	Score 2.991: Start Index [1269] with window size [5]

	Score 3.179: Start Index [1268] with window size [5]

	Score 3.364: Start Index [1334] with window size [5]

	Score 3.434: Start Index [1397] with window size [5]

	Score 3.644: Start Index [1266] with window size [5]

	Score 3.65: Start Index [689] with window size [5]

	Score 3.726: Start Index [222] with window size [5]

>>  889637 : Estudo do estigma como barreira ao tratamento dos transtornos mentais no contexto de uma intervenção psicossocial : Total 10 

	Score 3.009: Start Index [77] with window size [4]

	Score 3.044: Start Index [25] with window size [1]

	Score 3.054: Start Index [75] with window size [1]

	Score 3.38: Start Index [14] with window size [3]

	Score 3.567: Start Ind

# Parte Extra: Sumarização

In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage

In [36]:
openai_api_key = "xxxxx"

In [37]:
chat = ChatOpenAI(
    model_name='gpt-3.5-turbo-16k',
    temperature = 0.2,
    openai_api_key = openai_api_key,         
    max_tokens=8000
)

In [38]:
# _id = "918768"
_id = "916013"

## Testes

### Sumarização do Artigo

In [41]:
texto = """
Sua tarefa é sumarizar trechos retirados de um artigo científico, acompanhado do resumo do artigo e seu título.
Com essas informações, você deve gerar uma sumarização sobre esse artigo e seu conteúdo, seu objetivo e suas possíveis descobertas.
A seguir, será passado o resumo e em seguida os trechos, todos declarados por colchetes.
Após isso, será dado o comando para iniciar sua atuação. 
Importante, retorne apenas a sumarização, sem informações extras e explicações sobre o que está sendo realizado. 
Isso não é um chat de conversa.

"""

title = metadata_dict[_id]["title"]
abstract = metadata_dict[_id]["abstract"]

texto += f"[TITULO]\n{title}\n\n"
texto += f"[RESUMO]\n{abstract}\n\n"

for doc in sorted(results_by_document[_id])[:10]:
    texto += f"[TRECHO]\n{doc[1].page_content}\n\n"
    
texto += "\n\nTrechos Finalizados. Retorne a sumarização do artigo como instruído e com 200 palavras:"

In [ ]:
print(texto)

In [ ]:
messages = [SystemMessage(content=texto)]

response = chat(messages)

In [ ]:
print(response.content)

### Sumarização do Artigo Contextualizada com a Busca

In [ ]:
query

In [ ]:
# texto = f"""
# A partir da busca "{query}" feita pelo usuário, foram recuperados trechos e o resumo do seguinte artigo similar.
# Sua tarefa é sumarizar esses textos do artigo contextualizando em relação a busca realizada.
# A sumarização será feita em quatro tópicos distintos:

# - Sumarização do artigo, com base no título, resumo e seus trechos.
# - Sumarização do objetivo do artigo.
# - Seleção das principais informações sobre o artigo, como descobertas, conclusões, etc.
# - Sumarização de como o artigo se relaciona com a busca realizada pelo usuário.

# A seguir, será passado o resumo e em seguida os trechos, todos declarados por colchetes.
# Após isso, será dado o comando para iniciar sua atuação. 
# Importante, retorne apenas a sumarização, sem informações extras e explicações sobre o que está sendo realizado. 
# Isso não é um chat de conversa.

# """

# title = metadata_dict[_id]["title"]
# abstract = metadata_dict[_id]["abstract"]

# texto += f"[TITULO]\n{title}\n\n"
# texto += f"[RESUMO]\n{abstract}\n\n"

# for doc in results_by_document[_id][:10]:
#     texto += f"[TRECHO]\n{doc.page_content}\n\n"
    
# texto += "\n\nTrechos Finalizados. Retorne as sumarizações como instruído e com um tamanho médio (mais ou menos 300 palavras):"

In [ ]:
# texto = f"""
# A partir da busca "{query}" feita pelo usuário, foram recuperados trechos e o resumo do seguinte artigo similar.
# Sua tarefa é relacionar a busca realizada pelo usuário com o artigo, a partir do resumo e os trechos de similaridade. 
# Inclua informações importantes, como descobertas, conclusões, etc.

# A seguir, será passado o resumo e em seguida os trechos, todos declarados por colchetes.
# Após isso, será dado o comando para iniciar sua atuação. 
# Importante, retorne apenas a sumarização, sem informações extras e explicações sobre o que está sendo realizado. 
# Isso não é um chat de conversa.

# """

# title = metadata_dict[_id]["title"]
# abstract = metadata_dict[_id]["abstract"]

# texto += f"[TITULO]\n{title}\n\n"
# texto += f"[RESUMO]\n{abstract}\n\n"

# for doc in results_by_document[_id][:10]:
#     texto += f"[TRECHO]\n{doc.page_content}\n\n"
    
# texto += "\n\nTrechos Finalizados. Retorne as sumarizações como instruído e com um tamanho médio (mais ou menos 300 palavras):"

In [ ]:
texto = f"""
A partir da busca "{query}" feita pelo usuário, foram recuperados trechos e o resumo do seguinte artigo similar.
A seguir, será passado o resumo e em seguida os trechos, todos declarados por colchetes.
Após isso, será dado o comando para iniciar sua atuação. 

"""

title = metadata_dict[_id]["title"]
abstract = metadata_dict[_id]["abstract"]

texto += f"[TITULO]\n{title}\n\n"
texto += f"[RESUMO]\n{abstract}\n\n"

for doc in results_by_document[_id][:10]:
    texto += f"[TRECHO]\n{doc.page_content}\n\n"
    
texto += """
Trechos Finalizados.
Sua resposta será no formato JSON com 3 propriedades que respondem:
1. "sumarizacao": Sumarize o artigo a partir do resumo e os trechos com as informações mais relevantes sobre ele, como descobertas, etc. (200 palavras)
2. "objetivo_conclusao": Sumarize o ojetivo e a conclusão do artigo. (200 palavras)
3. "relevancia": Disserte sobre como o artigo encontrado se relaciona com a busca realizada e como o usuário poderá utilizar para auxiliar no estudo do tema. (200 palavras)
"""

In [ ]:
print(texto)

In [ ]:
messages = [SystemMessage(content=texto)]

response = chat(messages)

In [ ]:
print(response.content)

In [ ]:
json.loads(response.content)

# Parte 3: Filtragem e Reranking

## Reranker Class

In [122]:
import cohere

class Reranker:
    
    def __init__(self, cohere_key, model):
        self.cohere_client = cohere.Client(cohere_key)
        self.model = model
    
    def prepare_documents(self, metadata_dict: dict, retrieved_documents: dict):
        """
        - metadata_dict: all metadata
        - retrieved_documents: retrieved sentence documents grouped by key
        """
        
        documents = {}
        for key in retrieved_documents:
            documents[key] = metadata_dict[key]
            documents[key]["sentences"] = [doc[1] for doc in retrieved_documents[key]]   # score and doc tuple
        
        return documents
        
    
    def encode_document(self, document: dict):
        sentences_text = [doc.page_content for doc in document["sentences"]]
        sentences = "\n\n".join(sentences_text)
        text = document['title'] + "\n\n" + document['abstract'] + "\n\n" + sentences
        return text
    
    def rerank(self, query: str, documents: dict, top_n: int):
        """
        - query: string used by to retrieve documents
        - documents: dictionary with key as the article id and value as the metadata for the document, with title, abstract and sentences
        - top_n: number of top documents to return
        - return: list of document metadata
        """
        
        keys_list = list(documents.keys())
        document_list = [self.encode_document(documents[key]) for key in keys_list]
        
        response = self.cohere_client.rerank(
          model = self.model,
          query = query,
          documents = document_list,
          top_n = top_n
        )
        
        ordered_documents = []
        for res in response:
            key = keys_list[res.index]
            doc = documents[key]
            doc["relevance_score"] = res.relevance_score
            ordered_documents.append(doc)
        
        return ordered_documents

## Teste

In [123]:
query = buscas[9]
query

'estudo sobre o desenvolvimento de transtornos mentais em profissionais da medicina'

In [124]:
retrieved_documents = await retriever.search(query, k=1000, fetch_k=100000, max_k_by_document=8, filter={"type":"sentences"})

In [125]:
reranker = Reranker(cohere_key="xxxxxx", model="rerank-multilingual-v2.0")

In [126]:
documents = reranker.prepare_documents(metadata_dict, retrieved_documents)

In [130]:
rerankerd_documents = reranker.rerank(query, documents, 10)

In [132]:
rerankerd_documents[0]

{'title': 'Prevalência de transtornos mentais no estudantes de medicina da Universidade Federal do Rio de Janeiro',
 'authors': ['Campos Junior, Ailson',
  'Lima, Lúcia Abelha orient.',
  'Brasil, Marco Antônio coorient.',
  'Universidade Federal do Rio de Janeiro. Instituto de Estudos em Saúde Coletiva.'],
 'abstract': 'Introdução: A prevalência de transtornos mentais encontrada entre os estudantes de medicina brasileiros tem sido mais alta do que a encontrada em estudantes de países desenvolvidos. Apesar de ser consenso na literatura que as prevalências encontradas em países em desenvolvimento são mais elevadas, torna-se relevante conhecer como esta dinâmica e seus fatores de risco se apresentam na realidade dos estudantes médicos da UFRJ. O objetivo deste estudo foi investigar a prevalência e fatores associados ao episódio depressivo maior e transtorno de ansiedade generalizada em estudantes de medicina da Universidade Federal do Rio de Janeiro, bem como perceber como estes estudant

# Parte 4: Retrieval Augmented Generation

* Etapa 1: Retrieval - A partir da busca seleciona os principais trechos e resumos similares.
* Etapa 2: Agrupa as Informações - Unifica os resumos e trechos de um único artigo em um objeto.
* Etapa 3: Filtragem e Reranking - Para ser mais preciso na busca, filtra e reordena os artigos selecionados.
* Etapa 4: Generation - Com a informação de título, autores, resumo e trechos similares, gera os textos de sumarização.

## Generation

In [159]:
import ast, json


class Generator:
    
    def __init__(self, query, document_info: dict, top_k_results: int = 10, model_name='gpt-3.5-turbo-16k', temperature=0.2, max_tokens=10000):
        self.llm = ChatOpenAI(
            model_name=model_name,
            temperature=temperature,
            openai_api_key=openai_api_key,       
            max_tokens=max_tokens
        )
        
        self.query = query
        self.title = document_info["title"]
        self.abstract = document_info["abstract"]
        self.authors = document_info["authors"]
        self.search_results = document_info["sentences"]
        self.top_k_results = top_k_results
        
    async def run_llm(self):
        text = (
            f"""A partir da busca "{self.query}" feita pelo usuário, foram recuperados trechos e o resumo do seguinte artigo similar.\n"""
            """A seguir, será passado o resumo e em seguida os trechos, todos declarados por colchetes.\n"""
            """Após isso, será dado o comando para iniciar sua atuação."""
            """\n\n"""
            f"""[TITULO]\n{self.title}"""
            """\n\n"""
            f"""[RESUMO]\n{self.abstract}"""
            """\n\n"""
        )

        for doc in self.search_results[:self.top_k_results]:
            text += f"[TRECHO]\n{doc.page_content}\n\n"

        text += (
            """Trechos Finalizados. Sua resposta será no formato JSON com 3 propriedades que respondem:\n"""
            """1. "sumarizacao": Sumarize o artigo a partir do resumo e os trechos com as informações mais relevantes sobre ele, como descobertas, etc. (200 palavras)\n"""
            """2. "objetivo_conclusao": Sumarize o ojetivo e a conclusão do artigo. (200 palavras)\n"""
            """3. "relevancia": Disserte sobre como o artigo encontrado se relaciona com a busca realizada e como o usuário poderá utilizar para auxiliar no estudo do tema. (200 palavras)\n"""
        )
        
        try:
            messages = [SystemMessage(content=text)]
            response = self.llm(messages)
            content = ast.literal_eval(response.content)
        except Exception as e:
            print(response.content)
            raise(e)
        
        self.llm_query = text
        self.llm_summary = content["sumarizacao"]
        self.llm_objective_conclusion = content["objetivo_conclusao"]
        self.llm_relevance = content["relevancia"]

    @property
    def article_summary(self):
        if not hasattr(self, "llm_summary"):
            raise Exception("Summary not found. First execute .run_llm() method.")
        return self.llm_summary
    
    @property
    def article_objectives_and_conclusion(self):
        if not hasattr(self, "llm_objective_conclusion"):
            raise Exception("Objectives and Conclusions not found. First execute .run_llm() method.")
        return self.llm_objective_conclusion
    
    @property
    def article_relevance(self):
        if not hasattr(self, "llm_relevance"):
            raise Exception("Relevance not found. First execute .run_llm() method.")
        return self.llm_relevance

In [144]:
# generator = Generator(query, metadata_dict[_id], results_by_document[_id])

## Retrieve & Generate

In [145]:
query = buscas[9]
query

'estudo sobre o desenvolvimento de transtornos mentais em profissionais da medicina'

In [146]:
retrieved_documents = await retriever.search(query, k=1000, fetch_k=100000, max_k_by_document=8, filter={"type":"sentences"})

In [147]:
reranker = Reranker(cohere_key="xxxxxx", model="rerank-multilingual-v2.0")

In [148]:
documents = reranker.prepare_documents(metadata_dict, retrieved_documents)

In [149]:
reranked_documents = reranker.rerank(query, documents, 10)

In [150]:
# best document
best_document_info = reranked_documents[0]

In [160]:
generator = Generator(query, best_document_info)

In [161]:
await generator.run_llm()

In [162]:
generator.query

'estudo sobre o desenvolvimento de transtornos mentais em profissionais da medicina'

In [163]:
generator.title

'Prevalência de transtornos mentais no estudantes de medicina da Universidade Federal do Rio de Janeiro'

In [164]:
generator.authors

['Campos Junior, Ailson',
 'Lima, Lúcia Abelha orient.',
 'Brasil, Marco Antônio coorient.',
 'Universidade Federal do Rio de Janeiro. Instituto de Estudos em Saúde Coletiva.']

In [165]:
generator.abstract

'Introdução: A prevalência de transtornos mentais encontrada entre os estudantes de medicina brasileiros tem sido mais alta do que a encontrada em estudantes de países desenvolvidos. Apesar de ser consenso na literatura que as prevalências encontradas em países em desenvolvimento são mais elevadas, torna-se relevante conhecer como esta dinâmica e seus fatores de risco se apresentam na realidade dos estudantes médicos da UFRJ. O objetivo deste estudo foi investigar a prevalência e fatores associados ao episódio depressivo maior e transtorno de ansiedade generalizada em estudantes de medicina da Universidade Federal do Rio de Janeiro, bem como perceber como estes estudantes avaliam o curso. Métodos: Foi realizado um estudo transversal em uma amostra representativa e aleatória de 296 estudantes (taxa de participação=91,4%), utilizando os seguintes instrumentos: PHQ-9 (Patient Health Questionare-9) para avaliação do episódio depressivo maior, GAD-7 (General Anxiety Disorder-7) para avaliaç

In [166]:
generator.article_summary

'Este estudo investigou a prevalência e fatores associados ao episódio depressivo maior e transtorno de ansiedade generalizada em estudantes de medicina da Universidade Federal do Rio de Janeiro (UFRJ). Foi encontrada uma taxa de prevalência de episódio depressivo maior de 59,5% e uma taxa de prevalência de transtorno de ansiedade generalizada de 51,9%. Os fatores associados foram o gênero feminino, história de tratamento psiquiátrico atual e percepção negativa de estressores. Os estudantes avaliaram o ambiente educacional como tendo muitos problemas. Considerando esses resultados, é fundamental implementar programas institucionais que promovam a saúde e o desenvolvimento integral dos estudantes para melhorar sua qualidade de vida.'

In [167]:
generator.article_objectives_and_conclusion

'O objetivo deste estudo foi investigar a prevalência e fatores associados ao episódio depressivo maior e transtorno de ansiedade generalizada em estudantes de medicina da UFRJ, bem como avaliar a percepção dos alunos sobre o ambiente de ensino. Os resultados mostraram altas taxas de transtornos mentais nessa população, com fatores de risco como gênero feminino, história de tratamento psiquiátrico e percepção negativa de estressores. Além disso, os estudantes avaliaram o ambiente educacional como problemático. Conclui-se que é necessário implementar programas institucionais que promovam a saúde e o bem-estar dos estudantes para melhorar sua qualidade de vida.'

In [168]:
generator.article_relevance

'Esse artigo é relevante para a busca realizada, pois aborda o tema do desenvolvimento de transtornos mentais em profissionais da medicina, mais especificamente em estudantes de medicina da UFRJ. Os resultados encontrados mostram altas taxas de episódio depressivo maior e transtorno de ansiedade generalizada nessa população, além de fatores de risco associados. Essas informações podem ser úteis para profissionais da saúde, gestores educacionais e estudantes de medicina, pois destacam a importância de programas institucionais que promovam a saúde mental e o bem-estar dos estudantes. Além disso, os resultados sobre a percepção dos alunos sobre o ambiente de ensino podem fornecer insights para melhorias na qualidade do curso. Portanto, o usuário pode utilizar esse artigo como base para compreender a prevalência e os fatores de risco dos transtornos mentais em estudantes de medicina, bem como para embasar a implementação de programas de promoção da saúde nessa população.'